In [2]:
import pandas as pd
import numpy as np
import sklearn
import nltk
import re
import string

## Pivot Genre labels

In [3]:
movielist=pd.read_csv('./Data/ml-latest-small/movies.csv')
# get release year and clean film title
movielist['release_year']=movielist.title.apply(lambda x: x.strip()[-5:].replace(')', '') if x[-1]==')' else 9999 )
movielist['film_title']=movielist.apply(lambda x: x.title[:-6].strip() if x.release_year!=9999 else x.title , axis=1)
movielist['genres_raw']=movielist.genres.apply(lambda x: x.replace('|', ', '))
display(movielist.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movieId       9742 non-null   int64 
 1   title         9742 non-null   object
 2   genres        9742 non-null   object
 3   release_year  9742 non-null   object
 4   film_title    9742 non-null   object
 5   genres_raw    9742 non-null   object
dtypes: int64(1), object(5)
memory usage: 456.8+ KB


None

In [4]:
# define function to perform one-hot encoding (a column flag for each genre) and pivot data
def create_genre_flags(df, col_list, id_vars=['movieId']):
    print('Encoding the following genres: '+', '.join(col_list))
    output=df.copy()
    for each in col_list:
        output[each]=output.genres.apply(lambda x: 1 if each in x else 0)
    output=output.drop(['title', 'film_title', 'genres', 'genres_raw'], axis=1)
    output=pd.melt(output, id_vars=id_vars)
    output['attribute_type']='genre'
    return output

In [5]:
# get list of all genres in data
genres=list(set(np.concatenate(movielist.genres.str.split('|'))))

# create one-hot flag per genre and pivot the data (wide to long)
movie_genres_pivot=create_genre_flags(movielist, genres)
movie_genres_pivot.head()



Encoding the following genres: Adventure, Animation, Fantasy, Action, Sci-Fi, Mystery, Crime, IMAX, Horror, (no genres listed), War, Documentary, Thriller, Musical, Children, Comedy, Western, Drama, Romance, Film-Noir


,movieId,variable,value,attribute_type
0,1,release_year,1995,genre
1,2,release_year,1995,genre
2,3,release_year,1995,genre
3,4,release_year,1995,genre
4,5,release_year,1995,genre


In [5]:
movie_genres_pivot.shape

(204582, 4)

### User Tags: Create Tfidf vectors for each movie (combined user tags)

In [6]:
#load data: one row per tag per movie per userId
movietags=pd.read_csv('./Data/ml-latest-small/tags.csv')

print(len(movietags))
print(movietags.head())
display(movietags.info())

3683
   userId  movieId              tag   timestamp
0       2    60756            funny  1445714994
1       2    60756  Highly quotable  1445714996
2       2    60756     will ferrell  1445714992
3       2    89774     Boxing story  1445715207
4       2    89774              MMA  1445715200
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


None

In [7]:
# Starting with a per-movie focus rather than retaining any user information (e.g. content based rather than collaborative)
# not removing any duplicate tags, as the count of occurences could impact the tfidf value
movietags=pd.DataFrame(movietags.groupby('movieId')['tag'].apply(' '.join)).reset_index()
movietags=movietags.rename(columns={'tag': 'combined_tags'})
movietags.head()

,movieId,combined_tags
0,1,pixar pixar fun
1,2,fantasy magic board game Robin Williams game
2,3,moldy old
3,5,pregnancy remake
4,7,remake


In [ ]:
punctuation / lower case / stop words

In [67]:
#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_jobid = tfidf_vectorizer.fit_transform(movietags['combined_tags']) #fitting and transforming the vector

# concatenate data to include all word vectors per movieId
perMovieVectors=pd.DataFrame(tfidf_jobid.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
perMovieVectors=pd.concat([movietags.movieId, perMovieVectors], axis=1)
print(perMovieVectors.shape)
perMovieVectors.head()

(1572, 1745)


,movieId,06,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001,...,york,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
# Pivot data from wide to long, attribute type = tfidf
movie_tags_pivot=pd.melt(perMovieVectors, id_vars=['movieId'])
movie_tags_pivot['attribute_type']='Tfidf'
movie_tags_pivot.head()


,movieId,variable,value,attribute_type
0,1,06,0.0,Tfidf
1,2,06,0.0,Tfidf
2,3,06,0.0,Tfidf
3,5,06,0.0,Tfidf
4,7,06,0.0,Tfidf


In [6]:
movie_tags_pivot.shape

(2741568, 4)

# Scrape more text using imdb link

In [6]:
links = pd.read_csv('./Data/ml-latest-small/links.csv')
movielist=pd.merge(movielist, links, on='movieId', how='left')


#### Using tmDB api
https://developer.themoviedb.org/docs

In [1]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
tmdb = TMDb()
tmdb.api_key = '2837bf413a057f0237648c0dcd78d017'
tmdb.language = 'en'
tmdb.debug = True

movie = Movie()
# for example the movie with id=862
m = movie.details(862)
print (m.overview)

Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.


In [4]:
## 1000 rows took 10.5 mins

from tmdbv3api import TMDb
from tmdbv3api import Movie
tmdb = TMDb()
tmdb.api_key = '2837bf413a057f0237648c0dcd78d017'
tmdb.language = 'en'
tmdb.debug = True

movie = Movie()
def get_overview(tmdbMovie, id):
    try:
        out=tmdbMovie.details(id).overview
    except:
        out='NA'
        pass
    return out
mini=movielist[:1001].copy()
mini['description']=mini.tmdbId.apply(lambda x: get_overview(movie, x))
mini.to_csv('./Data/ml-latest-small/movie_links_text_1000.csv', index=False)
mini.head()


,movieId,title,genres,release_year,film_title,imdbId,tmdbId,description
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story,114709,862.0,"Led by Woody, Andy's toys live happily in his ..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,Jumanji,113497,8844.0,When siblings Judy and Peter discover an encha...
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,Grumpier Old Men,113228,15602.0,A family wedding reignites the ancient feud be...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,Waiting to Exhale,114885,31357.0,"Cheated on, mistreated and stepped on, the wom..."
4,5,Father of the Bride Part II (1995),Comedy,1995,Father of the Bride Part II,113041,11862.0,Just when George Banks has recovered from his ...


In [10]:
#check that exception works
get_overview(movie, 293894059718)

'NA'

In [7]:
# batches of 500
for each in range(1001, len(movielist), 500):
    mini=movielist[each:min(each+500, len(movielist))].copy()
    mini['description']=mini.tmdbId.apply(lambda x: get_overview(movie, x))
    print(f'rows {each} to {min(each+500, len(movielist))} complete')
    mini.to_csv(f'./Data/ml-latest-small/movie_links_text_{each}_to_{min(each+500, len(movielist))}.csv', index=False)

rows 1001 to 1501 complete
rows 1501 to 2001 complete
rows 2001 to 2501 complete
rows 2501 to 3001 complete
rows 3001 to 3501 complete
rows 3501 to 4001 complete
rows 4001 to 4501 complete
rows 4501 to 5001 complete
rows 5001 to 5501 complete
rows 5501 to 6001 complete
rows 6001 to 6501 complete
rows 6501 to 7001 complete
rows 7001 to 7501 complete
rows 7501 to 8001 complete
rows 8001 to 8501 complete
rows 8501 to 9001 complete
rows 9001 to 9501 complete
rows 9501 to 9742 complete


### Clean text & generate Tfidf features

In [8]:
# combine all description files then clean text etc
moviedesc=pd.read_csv('./Data/ml-latest-small/movie_links_text_1000.csv')
for each in range(1001, len(movielist), 500):
    mini=pd.read_csv(f'./Data/ml-latest-small/movie_links_text_{each}_to_{min(each+500, len(movielist))}.csv')
    moviedesc=pd.concat([moviedesc, mini], axis=0)

moviedesc=moviedesc.reset_index(drop=True)
moviedesc['description'] = moviedesc['description'].astype(str)
moviedesc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movieId       9742 non-null   int64  
 1   title         9742 non-null   object 
 2   genres        9742 non-null   object 
 3   release_year  9742 non-null   int64  
 4   film_title    9742 non-null   object 
 5   imdbId        9742 non-null   int64  
 6   tmdbId        9734 non-null   float64
 7   description   9742 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 609.0+ KB


In [99]:

#import nltk
#nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import word_tokenize


from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()
ps=PorterStemmer()

def black_txt(token):
    token=ps.stem(token)
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "",text)
  text=re.sub("(\\d|\\W)+"," ",text) 
  # text = text.replace("nbsp", "")
  clean_text = [ wn.lemmatize(word) for word in word_tokenize(text.lower()) if black_txt(word)]
  clean_text2 = [word for word in clean_text if black_txt(word)]
  return " ".join(clean_text2)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\helen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\helen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [100]:

moviedesc['clean_description'] = moviedesc['description'].apply(clean_txt)

#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=5)

tfidf_jobid = tfidf_vectorizer.fit_transform(moviedesc['clean_description']) #fitting and transforming the vector

# concatenate data to include all word vectors per movieId
moviedesc=pd.concat([moviedesc, pd.DataFrame(tfidf_jobid.toarray(), columns=tfidf_vectorizer.get_feature_names_out())], axis=1)
print(moviedesc.shape)



(9742, 6945)


In [101]:
feature_names = np.array(tfidf_vectorizer.get_feature_names_out() )
top_term=feature_names[tfidf_jobid.argmax(axis=1)]
top_term

array([['andys'],
       ['game'],
       ['max'],
       ...,
       ['flint'],
       ['user'],
       ['sketch']], dtype=object)

In [102]:
pd.options.display.max_colwidth=False
moviedesc.clean_description[:5]

0    led woody andys toy live happily room andys birthday brings buzz lightyear onto scene afraid losing place andys heart woody plot buzz circumstance separate buzz woody owner duo eventually learns put aside difference                                                  
1    sibling judy peter discover enchanted board game open door magical world unwittingly invite alan adult trapped inside game year living room alans only hope freedom finish game prof risky three find themselves running giant rhinoceros evil monkey terrifying creature
2    family wedding reignites ancient feud next door neighbor fishing buddy john max meanwhile sultry italian divorcée open restaurant local bait shop alarming local worry shell scare fish away interested seafood cooking hot time max                                     
3    cheated mistreated stepped woman holding breath waiting elusive good man break string stellar lover friend confidant vannah bernie glo robin talk determined find better way breathe  

In [108]:
tfarray=tfidf_jobid[:5].toarray()
feature_names[np.argsort(-tfarray, axis=-1)[:, :15]]

array([['andys', 'buzz', 'woody', 'afraid', 'aside', 'toy', 'separate',
        'onto', 'happily', 'difference', 'circumstance', 'duo',
        'birthday', 'losing', 'room'],
       ['game', 'risky', 'enchanted', 'judy', 'monkey', 'alan', 'finish',
        'unwittingly', 'sibling', 'invite', 'board', 'freedom', 'prof',
        'terrifying', 'running'],
       ['max', 'local', 'divorcée', 'cooking', 'feud', 'scare', 'worry',
        'interested', 'fishing', 'fish', 'shell', 'restaurant',
        'italian', 'hot', 'shop'],
       ['stepped', 'cheated', 'bernie', 'elusive', 'holding', 'string',
        'robin', 'waiting', 'talk', 'better', 'lover', 'determined',
        'break', 'good', 'way'],
       ['george', 'expecting', 'selling', 'planning', 'receives',
        'arrival', 'news', 'pregnant', 'wedding', 'bank', 'kid',
        'change', 'like', 'plan', 'daughter']], dtype=object)

In [16]:
# Pivot data from wide to long, attribute type = description tfidf
moviedescpivot=pd.melt(moviedesc.drop(columns=[ 'title','genres','release_year','film_title','imdbId','tmdbId','description','clean_description']).rename(columns={'value':'"value"'}), id_vars=['movieId'])
moviedescpivot['attribute_type']='description_Tfidf'
moviedescpivot.head(), moviedescpivot.shape


(   movieId variable  value     attribute_type
 0        1    aaron    0.0  description_Tfidf
 1        2    aaron    0.0  description_Tfidf
 2        3    aaron    0.0  description_Tfidf
 3        4    aaron    0.0  description_Tfidf
 4        5    aaron    0.0  description_Tfidf,
 (67619222, 4))

In [29]:
from sklearn.decomposition import PCA
n_components=50
pca_2=PCA(n_components=n_components)
pca_20_output=pca_2.fit_transform(moviedesc.drop(columns=[ 'movieId','title','genres','release_year','film_title','imdbId','tmdbId','description','clean_description']).to_numpy())
print(pca_2.explained_variance_ratio_.sum() )
pca_pivot=pd.concat([ moviedesc[['movieId']], pd.DataFrame(pca_20_output, columns=range(n_components))], axis=1)
print(pca_pivot.shape)
pca_pivot=pd.melt(pca_pivot, id_vars=['movieId'])
pca_pivot['attribute_type']='description_Tfidf_PCA'
pca_pivot.head(),pca_pivot.shape


0.08986935330832742
(9742, 51)


(   movieId variable     value         attribute_type
 0        1        0 -0.006099  description_Tfidf_PCA
 1        2        0 -0.014466  description_Tfidf_PCA
 2        3        0 -0.011223  description_Tfidf_PCA
 3        4        0 -0.014349  description_Tfidf_PCA
 4        5        0 -0.011945  description_Tfidf_PCA,
 (487100, 4))

In [ ]:
pca_pivot

In [ ]:
moviedesc.to_csv('./Data/ml-latest-small/movie_description_tfidf.csv', index=False)

# Combine data

### Movie Fact table

In [9]:
# check the three datasets which will be combined into movie fact: 
movielist.movieId.nunique(), movietags.movieId.nunique(), moviedesc.movieId.nunique()

(9742, 1572, 9742)

In [10]:
moviedesc['description'] = moviedesc['description'].astype(str)

#create fact table for movide ID, film title, raw genre list and combined tags text (latter two for traceability)
tableau_movie_list=pd.merge(movielist[['movieId', 'film_title','release_year', 'genres_raw']],movietags, on='movieId', how='left' )

tableau_movie_list=pd.merge( tableau_movie_list,moviedesc[['movieId', 'description']], on='movieId')
tableau_movie_list.head()
tableau_movie_list.to_csv('../My Tableau Repository/Data sources/Movies/movie_fact.csv', index=False)

### Attributes table (Long)

In [30]:
tabeau_movie_dataset=pd.concat([movie_genres_pivot,pca_pivot])  #movie_tags_pivot, moviedescpivot
tabeau_movie_dataset=tabeau_movie_dataset.rename(columns={'variable':'attribute'})
tabeau_movie_dataset.head()
tabeau_movie_dataset.shape

(691682, 4)

In [31]:
tabeau_movie_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 691682 entries, 0 to 487099
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   movieId         691682 non-null  int64 
 1   attribute       691682 non-null  object
 2   value           691682 non-null  object
 3   attribute_type  691682 non-null  object
dtypes: int64(1), object(3)
memory usage: 26.4+ MB


In [22]:
tableau_movie_list.shape, tabeau_movie_dataset.shape

NameError: name 'tableau_movie_list' is not defined

In [32]:
# Save to Tableau data repository
tabeau_movie_dataset.to_csv('../My Tableau Repository/Data sources/Movies/movie_pivot_genres_and_descpca.csv', index=False)
#tableau_movie_list.to_csv('../My Tableau Repository/Data sources/Movies/movie_fact.csv', index=False)

In [16]:
movielist.movieId.nunique(), movietags.movieId.nunique()

(9742, 1572)